In [1]:
import numpy as np
import pandas as pd
from importlib import reload
import datetime

import shared
import config
import provider_yfinance as provider

reload(shared)
reload(config)
reload(provider)

cfg = config.get_config('^GDAXI')

# overwrite download_end_dt: use cached data
# config.overwrite_end_dt(cfg, '2019-12-19')
# config.save_config(cfg)

# model sav

config> created config from file: './config.json'
config> config
        - base:
            - config_file_path: /mnt/c/notebooks/sandbox/config.json
        - datasets:
            - stocks: 30
            - benchmarks: 69
        - prepare:
            - data_start_dt: 2018-02-09
            - data_end_dt: 2020-01-07
            - cache_dir: /mnt/c/notebooks/sandbox/cache/20200107/
        - train:            
            - window_trading_days: [3, 5, 21, 35, 50]
            - lag_trading_days: [1, 2, 3, 4, 5]
            - label_max_high_weight: 3.0
            - label_max_close_weight: 1.0
            - settings: 12
        - model:
            - max_samples: 40
            - batch_size: 200
            - lstm_hidden_size: 256
            - early_stopping_patience: 10
            - validaion_monitor: val_mean_squared_error
            - max_epochs: 1000
            - base_dir: /mnt/c/notebooks/sandbox/model/20200110/            
        


In [2]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense, BatchNormalization, Masking
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K

import pathlib
import munch
import pickle
import numpy as np
import pandas as pd


Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [ ]:
def load_weights(cfg, submodel_settings, model, ticker_name=''):
    pth_submodel = pathlib.Path(f"{cfg.model.base_dir}/{submodel_settings.id}/{ticker_name}")
    f_model_weights = pth_submodel.joinpath(cfg.model.model_weights_file_name)
    f_optimizer_weights = pth_submodel.joinpath(cfg.model.optimizer_weights_file_name)
    shared.mkdirs(pth_submodel)
    if f_model_weights.is_file():
        try:
            model.load_weights(f_model_weights)
            print(f"model> loaded model weights from '{f_model_weights.resolve()}'")
        except ValueError as e:
            print(f"WARN model> failed to load model weights from '{f_model_weights.resolve()}': ${e}")
    if f_optimizer_weights.is_file():
        model._make_train_function()
        try:
            with open(f_optimizer_weights.resolve(), 'rb') as f:
                model.optimizer.set_weights(pickle.load(f))    
                print(f"model> loaded optimizer weights from '{f_optimizer_weights.resolve()}'")
        except ValueError as e:
            print(f"WARN model> failed to load optimizer weights from '{f_optimizer_weights.resolve()}': ${e}")
    
def save_weights(cfg, submodel_settings, model, ticker_name=''):
    print(f"model> trying to save weights ...") 
    pth_submodel = pathlib.Path(f"{cfg.model.base_dir}/{submodel_settings.id}/{ticker_name}")
    f_model_weights = pth_submodel.joinpath(cfg.model.model_weights_file_name)
    f_optimizer_weights = pth_submodel.joinpath(cfg.model.optimizer_weights_file_name)
    shared.mkdirs(pth_submodel)
    model.save_weights(f_model_weights)
    print(f"model> saved model weights to '{f_model_weights.resolve()}'")
    with open(f_optimizer_weights.resolve(), 'wb') as f:
        pickle.dump(K.batch_get_value(getattr(model.optimizer, 'weights')), f)
        print(f"model> saved optimizer weights to '{f_optimizer_weights.resolve()}'")

def create_model(cfg, submodel_settings, model_data, ticker_name=''):
    num_samples = model_data.shape[0]
    num_features = len(model_data.X.head(1).tolist()[0][0][0][0])
    input_length = submodel_settings.lookback_days
    input_dim = num_features
    lstm_dim = cfg.model.lstm_hidden_size
    output_dim = 1
    model = Sequential()
    model.add(BatchNormalization(input_shape=(input_length, input_dim)))
    model.add(Masking())    
    model.add(LSTM(lstm_dim, dropout=.2, recurrent_dropout=.2, return_sequences=True, activation="softsign"))
    model.add(LSTM(lstm_dim, dropout=.2, recurrent_dropout=.2, return_sequences=True, activation="softsign"))
    model.add(LSTM(lstm_dim, dropout=.2, recurrent_dropout=.2, activation="softsign"))
    model.add(Dense(output_dim))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error', 'mean_squared_error'])
    print(f'model> model created\n:{model.summary()}')
    load_weights(cfg, submodel_settings, model, ticker_name)
    return model

def train_model(cfg, submodel_settings, model, model_data, ticker_name=''):
    num_samples = model_data.shape[0]
    num_features = len(model_data.X.head(1).tolist()[0][0][0][0])
    input_length = submodel_settings.lookback_days
    input_dim = num_features    
    output_dim = 1
    X = np.hstack(np.asarray(model_data.X)).reshape(num_samples, input_length, input_dim)
    y = np.hstack(np.asarray(model_data.y)).reshape(num_samples, output_dim)
    pth_submodel = f"{cfg.model.base_dir}/{submodel_settings.id}/{ticker_name}"
    shared.mkdirs(pth_submodel)
    monitor = cfg.model.validaion_monitor
    patience = cfg.model.early_stopping_patience
    fit_params = {
        "batch_size": cfg.model.batch_size,
        "epochs": cfg.model.max_epochs,
        "verbose": 1,
        "validation_split": 0.1,
        "shuffle": True,
        "callbacks": [
            EarlyStopping(verbose=True, patience=patience, monitor=monitor),
            ModelCheckpoint(f"{pth_submodel}/best_weights_lstm-{cfg.model.lstm_hidden_size}_epoch-{{epoch:02d}}_val-{{{monitor}:.4f}}.hdf5", monitor=monitor, verbose=1, save_best_only=True)
        ]
    }
    print('model> fitting ... (Hit CTRL-C to stop early)')
    history = None
    try:
        history = model.fit(X, y, **fit_params)
    except KeyboardInterrupt:
        print('model> training stopped early!')
        history = model.history        
    save_weights(cfg, submodel_settings, model, ticker_name)
    return history

monitor = cfg.model.validaion_monitor
patience = cfg.model.early_stopping_patience
for submodel_settings in cfg.train.settings:
    print(f"sm-{submodel_settings.id}> training submodel ...")
    model_data = provider.prepare_submodel_data(cfg, submodel_settings)
    model = create_model(cfg, submodel_settings, model_data)
    history = train_model(cfg, submodel_settings, model, model_data)
    print(f"sm-{submodel_settings.id}> overall-{monitor} (best epoch): {history.history[monitor][np.max(history.epoch)-patience]}")
    print(f"sm-{submodel_settings.id}> overall-{monitor} (+-5 around best epoch): {np.mean(history.history[monitor][(np.max(history.epoch)-patience-5):(np.max(history.epoch)-patience+5)])}")
    for ticker_name in model_data.ticker.unique().tolist():
        ticker_data = model_data[model_data.ticker==ticker_name]
        model = create_model(cfg, submodel_settings, ticker_data, ticker_name)
        history = train_model(cfg, submodel_settings, model, ticker_data, ticker_name)
        print(f"sm-{submodel_settings.id}> {ticker_name}-{monitor} (best epoch): {history.history[monitor][np.max(history.epoch)-patience]}")
        print(f"sm-{submodel_settings.id}> {ticker_name}-{monitor} (+-5 around best epoch): {np.mean(history.history[monitor][(np.max(history.epoch)-patience-5):(np.max(history.epoch)-patience+5)])}")        
        
# single stock
# all stocks

sm-nr_1-lookback_3-label_1> training submodel ...




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 3, 1319)           5276      
_________________________________________________________________
masking_1 (Masking)          (None, 3, 1319)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 256)            1613824   
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 256)            525312    
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
__________

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 3, 1319)           5276      
_________________________________________________________________
masking_2 (Masking)          (None, 3, 1319)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 3, 256)            1613824   
_________________________________________________________________
lstm_5 (LSTM)                (None, 3, 256)            525312    
_________________________________________________________________
lstm_6 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 2,669,981
Trainable params: 2,667,343
Non-trainable params: 2,638
______________________________________________________________

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 3, 1319)           5276      
_________________________________________________________________
masking_3 (Masking)          (None, 3, 1319)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 3, 256)            1613824   
_________________________________________________________________
lstm_8 (LSTM)                (None, 3, 256)            525312    
_________________________________________________________________
lstm_9 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 2,669,981
Trainable params: 2,667,343
Non-trainable params: 2,638
______________________________________________________________

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 3, 1319)           5276      
_________________________________________________________________
masking_4 (Masking)          (None, 3, 1319)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 3, 256)            1613824   
_________________________________________________________________
lstm_11 (LSTM)               (None, 3, 256)            525312    
_________________________________________________________________
lstm_12 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 2,669,981
Trainable params: 2,667,343
Non-trainable params: 2,638
______________________________________________________________

In [ ]:
print(f"model> {monitor}(+-5 around best epoch): {np.mean(history.history[monitor][(np.max(history.epoch)-patience-5):(np.max(history.epoch)-patience+5)])}")


In [ ]:
df_test = pd.DataFrame({
    'X': pd.Series(model.predict(X_train).flatten()),
    'y': y_train.flatten()
})
df_test['diff'] = df_test.X - df_test.y
df_test

https://stackoverflow.com/questions/39674713/neural-network-lstm-input-shape-from-dataframe
https://stackoverflow.com/questions/49803503/lstm-preprocessing-build-3d-arrays-from-pandas-data-frame-based-on-id